## COVID 19 project : Week 1 Assignment

DATA SOURCES:

<div>
<img src="https://bryantarchway.com/wp-content/uploads/2020/04/corona.png" height="100" width="350"/>
</div>

World: https://www.worldometers.info/coronavirus/

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ef/COVID-19_Outbreak_Cases_in_India.svg/1200px-COVID-19_Outbreak_Cases_in_India.svg.png" height="100" width="150"/>

India: https://www.mohfw.gov.in/

#### Importing Packages

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#### Initialising URLs for World and India

In [75]:
world_url = "https://www.worldometers.info/coronavirus/"
who_url = "https://www.who.int/emergencies/diseases/novel-coronavirus-2019"
corona_demographic_url = "https://www.worldometers.info/coronavirus/coronavirus-age-sex-demographics/"
india_url = 'https://www.mohfw.gov.in/'

#### Validate Conectivity to the web-portals

In [76]:
def connectivity_checker(url):
    if requests.get(url).status_code == 200:
        return('Success!')
    elif requests.get(url).status_code == 404:
        return('Not Found.')
    
print("Connecting to Ministry of Health & Family Welfare Portal...")
print("Status: ",connectivity_checker(india_url))
print("Connecting to Worldometers Portal...")
print("Status: ",connectivity_checker(world_url))
print("Connecting to Worldometers Corona Demographics Portal...")
print("Status: ",connectivity_checker(corona_demographic_url))
print("Connecting to WHO Portal...")
print("Status: ",connectivity_checker(who_url))

Connecting to Ministry of Health & Family Welfare Portal...
Status:  Success!
Connecting to Worldometers Portal...
Status:  Success!
Connecting to Worldometers Corona Demographics Portal...
Status:  Success!
Connecting to WHO Portal...
Status:  Success!


### WEB SCRAPPING INDIA DATA
<div>
<img src="https://surroundvision.co.uk/wp-content/uploads/2018/05/Portfolio-Incredible-India-Banner-1920x900.jpg" height="100" width="550"/>
</div>

#### Scrapping Data

In [47]:
soup=BeautifulSoup(requests.get(india_url).content, "html.parser")
div_container = soup.find_all("div",class_="data-table table-responsive")
data_table_body = div_container[0].table.tbody    
tag = data_table_body.findAll('td')

In [48]:
confirmed_list = []
cured_list = []
deaths_list = []
states_list = []

def data_extractor(data_list,llimit,interval):
    for data_cntr in range(llimit,160,interval): data_list.append(tag[data_cntr].text) 
    return data_list

states = data_extractor(states_list,1,5)
confirmed_cases = data_extractor(confirmed_list,2,5)
cured_cases = data_extractor(cured_list,3,5)
death_cases = data_extractor(deaths_list,4,5)


#### Exporting data into dataframe:

In [50]:
df_india = pd.DataFrame(list(zip(states, confirmed_cases,cured_cases,death_cases)), columns=["States","Confirmed_Cases","Cured_Cases","Death_Cases"])
df_india.head(5)

,States,Confirmed_Cases,Cured_Cases,Death_Cases
0,Andaman and Nicobar Islands,33,16,0
1,Andhra Pradesh,1463,403,33
2,Arunachal Pradesh,1,1,0
3,Assam,42,29,1
4,Bihar,426,82,2


#### Data Cleaning

In [515]:
def missing_val_updater(dfname):
    """
    Replacing missing values with Zero
    """
    dfname.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    dfname.replace(np.nan,0, inplace=True) 
    return dfname

def detect_missing_val(df):
    """
    Detecting missing value
    """
    tot_col = len(df.columns)
    for i in range(1,tot_col):
        if df[df[df.columns[i]].isna()][df.columns[i]].shape[0] == 0:
            missing_val_updater(df)
            print("Replaced missing values with Zero for :",df.columns[i])
        else:
            print("No Missing Values for :",df.columns[i])

detect_missing_val(df_india)

Replaced missing values with Zero for : Confirmed_Cases
Replaced missing values with Zero for : Cured_Cases
Replaced missing values with Zero for : Death_Cases


### WEB SCRAPPING WORLD DATA

<div>
<img src="http://www.pngmart.com/files/11/Earth-Travel-World-PNG-Transparent-Image.png" height="100" width="750"/>
</div>

#### Scrapping Data

In [51]:
soup=BeautifulSoup(requests.get(world_url).content, "html.parser")

In [52]:
coronatable=soup.find_all("table")
co=coronatable[0]
country=[]
total_cases=[]
new_cases=[]
rows=co.find_all("tr")[9:-8]
for row in rows:
    col=row.find_all("td")
    country.append(col[0].text.strip())
    total_cases.append(col[1].text.strip())
    new_cases.append(col[2].text.strip())

#### Exporting raw data into dataframe:

In [53]:
raw_df=pd.DataFrame(list(zip(country, total_cases,new_cases)), columns=["Country","Total_Cases","New_Cases"])
raw_df.head()

,Country,Total_Cases,New_Cases
0,USA,"1,121,712","+26,689"
1,Spain,"242,988","+3,648"
2,Italy,"207,428","+1,965"
3,UK,"177,454","+6,201"
4,France,"167,346",+168


#### Data Cleaning

In [58]:
def missing_val_updater(dfname):
    """
    Replacing missing values with Zero
    """
    dfname.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    dfname.replace(np.nan,0, inplace=True)
    return dfname

def records_cleaner(val):
    """
    Converting strings to integers for numeric manipulation. 
    Removing symbols "+" and ","
    """
    records_list=[]
    for i in df_world[val]:
        s=str(i)
        s1=s.replace("+","")
        s2=s1.replace(",","")
        s3=int(s2)
        records_list.append(s3)
    return records_list


df_world = missing_val_updater(raw_df)
updated_total_cases = records_cleaner("Total_Cases")
updated_new_cases = records_cleaner("New_Cases")
df_world=pd.DataFrame(list(zip(country, updated_total_cases,updated_new_cases)), columns=["Country","Total_Cases","New_Cases"])
df_world.head()


,Country,Total_Cases,New_Cases
0,USA,1121712,26689
1,Spain,242988,3648
2,Italy,207428,1965
3,UK,177454,6201
4,France,167346,168


#### Demographics : Segmentation based on Age and types of medical conditions

In [462]:
print("Connecting to Worldometers Corona Demographics Portal...")
print("Status: ",connectivity_checker(corona_demographic_url))

soup=BeautifulSoup(requests.get(corona_demographic_url).content, "html.parser")
div_container = soup.find_all("table",class_="table table-hover table-bordered table-condensed table-list")
table_body = div_container[0]
tr_tag = table_body.findAll('tr')
deaths_list = []
share_deaths_list = []
age_list = []
pre_ex_cond_list = []
w_pre_ex_cond_list = []
unknwn_pre_ex_cond_list = []
total = []

for i in range(1,6): age_list.append(tr_tag[i].td.div.text)
limit = len(age_list)
for x in range(0,limit): age_list[x] = age_list[x].replace("years old", "").strip()   
for y in range(1,limit):
    td_tag = tr_tag[y].findAll("td")
    deaths_list.append(td_tag[1].text)
    share_deaths_list.append(td_tag[2].text)
    pre_ex_cond_list.append(td_tag[3].text)
    w_pre_ex_cond_list.append(td_tag[4].text)
    unknwn_pre_ex_cond_list.append(td_tag[5].text)

def data_cleaner(data_list,dtype):
    for x in range(0,len(data_list)): data_list[x] = dtype(data_list[x].replace(",","").replace("%",""))
    return data_list


u_death_list = data_cleaner(deaths_list,int)
u_share_deaths_list = data_cleaner(share_deaths_list,float)
u_pre_ex_cond_list = data_cleaner(pre_ex_cond_list,int)
u_w_pre_ex_cond_list = data_cleaner(w_pre_ex_cond_list,float)
u_unknwn_pre_ex_cond_list = data_cleaner(unknwn_pre_ex_cond_list,float)


agewise_df = pd.DataFrame(list(zip(age_list,u_death_list, u_share_deaths_list,u_pre_ex_cond_list,u_w_pre_ex_cond_list,u_unknwn_pre_ex_cond_list)), columns=["Age Groups","Deaths","Share of Deaths(%)","PC","WPC","UPC"])
agewise_df


Connecting to Worldometers Corona Demographics Portal...
Status:  Success!


,Age Groups,Deaths,Share of Deaths(%),PC,WPC,UPC
0,0 - 17,3,0.04,3,0.0,0.0
1,18 - 44,309,4.50,244,25.0,40.0
2,45 - 64,1581,23.10,1343,59.0,179.0
3,65 - 74,1683,24.60,1272,26.0,385.0


#### TOTAL: Age-wise Segmentation

In [430]:
tot_tag = tr_tag[6].findAll("td")
u_total_deaths = int(tot_tag[1].text.replace(",","").replace("%",""))
u_share_of_deaths = int(tot_tag[2].text.replace(",","").replace("%",""))
u_TPC = int(tot_tag[3].text.replace(",","").replace("%",""))
u_TWPC = int(tot_tag[4].text.replace(",","").replace("%",""))
u_TUPC = int(tot_tag[5].text.replace(",","").replace("%",""))

total_agewise_df = pd.DataFrame([[u_total_deaths, u_share_of_deaths,u_TPC,u_TWPC,u_TUPC]], columns=["Total Deaths","Total Share of Deaths(%)","TPC","TWPC","TUPC"])
total_agewise_df


,Total Deaths,Total Share of Deaths(%),TPC,TWPC,TUPC
0,6839,100,5151,137,1551


#### Demographics : Segmentation based on Gender and types of medical conditions

In [466]:
print("Connecting to Worldometers Corona Demographics Portal...")
print("Status: ",connectivity_checker(corona_demographic_url))

soup=BeautifulSoup(requests.get(corona_demographic_url).content, "html.parser")
div_container = soup.find_all("table",class_="table table-hover table-bordered table-condensed table-list")
table_body = div_container[1]
table_body
tr_tag = table_body.findAll('tr')
tr_tag
deaths_list = []
share_deaths_list,share_pre_ex_cond_list,share_w_pre_ex_cond_list,share_unknwn_pre_ex_cond_list = [],[],[],[]
gender_list = []
pre_ex_cond_list = []
w_pre_ex_cond_list = []
unknwn_pre_ex_cond_list = []


for i in range(1,3): gender_list.append(tr_tag[i].td.div.text)

limit = len(gender_list)
 
for y in range(1,limit+1):
    td_tag = tr_tag[y].findAll("td")
    deaths_list.append(td_tag[1].text)
    share_deaths_list.append(td_tag[2].text)
    pre_ex_cond_list.append(td_tag[3].text)
    share_pre_ex_cond_list.append(td_tag[4].text)
    w_pre_ex_cond_list.append(td_tag[5].text)
    share_w_pre_ex_cond_list.append(td_tag[6].text)
    unknwn_pre_ex_cond_list.append(td_tag[7].text)
    share_unknwn_pre_ex_cond_list.append(td_tag[8].text)

def data_cleaner(data_list,dtype):
    for x in range(0,len(data_list)): data_list[x] = dtype(data_list[x].replace(",","").replace("%",""))
    return data_list


u_death_list = data_cleaner(deaths_list,int)
u_share_deaths_list = data_cleaner(share_deaths_list,float)
u_pre_ex_cond_list = data_cleaner(pre_ex_cond_list,float)
u_w_pre_ex_cond_list = data_cleaner(w_pre_ex_cond_list,float)
u_unknwn_pre_ex_cond_list = data_cleaner(unknwn_pre_ex_cond_list,float)
u_share_unknwn_pre_ex_cond_list = data_cleaner(share_unknwn_pre_ex_cond_list,float)
u_share_w_pre_ex_cond_list = data_cleaner(share_w_pre_ex_cond_list,float)
u_share_pre_ex_cond_list = data_cleaner(share_pre_ex_cond_list,float)




genderwise_df = pd.DataFrame(list(zip(gender_list,u_death_list, u_share_deaths_list,u_pre_ex_cond_list,u_share_pre_ex_cond_list,u_w_pre_ex_cond_list,u_share_w_pre_ex_cond_list,u_unknwn_pre_ex_cond_list,u_share_unknwn_pre_ex_cond_list)), columns=["Gender","Deaths","Share_of_Deaths(%)","PC","Share_of_PC(%)","WPC","Share_of_WPC(%)","UPC","Share_of_UPC(%)"])
genderwise_df


Connecting to Worldometers Corona Demographics Portal...
Status:  Success!


,Gender,Deaths,Share_of_Deaths(%),PC,Share_of_PC(%),WPC,Share_of_WPC(%),UPC,Share_of_UPC(%)
0,Male,4095,618.0,3087.000,622.0,96.0,72.2,912.0,59.5
1,Female,2530,38.2,1.873,37.8,37.0,27.8,620.0,40.5


#### Demographics : Probability of Death by Age

In [498]:
soup=BeautifulSoup(requests.get(corona_demographic_url).content, "html.parser")
div_container = soup.find_all("table",class_="table table-hover table-bordered table-condensed table-list")
table_body = div_container[2]
table_body
tr_tag = table_body.findAll('tr')

age_list = []
death_rate_confirmed_list = []
death_rate_all_cases_list = []

limit = len(tr_tag)
for i in range(1,limit): age_list.append(tr_tag[i].td.text)
 
for y in range(1,limit):
    td_tag = tr_tag[y].findAll("td")
    death_rate_confirmed_list.append(td_tag[1].text)
    death_rate_all_cases_list.append(td_tag[2].text)

deathrate_df = pd.DataFrame(list(zip(age_list,death_rate_confirmed_list, death_rate_all_cases_list)), columns=["Age","Death Rate (Confirmed)","Death Rate (All Cases)"])
deathrate_df

,Age,Death Rate (Confirmed),Death Rate (All Cases)
0,80+ years old,21.9%,14.8%
1,70-79 years old,,8.0%
2,60-69 years old,,3.6%
3,50-59 years old,,1.3%
4,40-49 years old,,0.4%
5,30-39 years old,,0.2%
6,20-29 years old,,0.2%
7,10-19 years old,,0.2%
8,0-9 years old,,no fatalities


In [511]:
def detect_missing_val(df):
    tot_col = len(df.columns)
    print(tot_col)
    for i in range(1,tot_col):
        if df[df[df.columns[i]].isna()][df.columns[i]].shape[0] == 0:
            print("Missing Values detected for column : ",df.columns[i])
            missing_val_updater(df)
            print("Replaced missing values with Zero")
        else:
            print("No Missing Values")

detect_missing_val(deathrate_df)

3
Missing Values detected for column :  Death Rate (Confirmed)
Replaced missing values with Zero
Missing Values detected for column :  Death Rate (All Cases)
Replaced missing values with Zero


In [513]:
deathrate_df

,Age,Death Rate (Confirmed),Death Rate (All Cases)
0,80+ years old,21.9%,14.8%
1,70-79 years old,0,8.0%
2,60-69 years old,0,3.6%
3,50-59 years old,0,1.3%
4,40-49 years old,0,0.4%
5,30-39 years old,0,0.2%
6,20-29 years old,0,0.2%
7,10-19 years old,0,0.2%
8,0-9 years old,0,no fatalities


In [522]:
def records_cleaner(val,dtype):
    """
    Converting strings to integers for numeric manipulation. 
    Removing symbols and redundant strings
    """
    records_list=[]
    for i in deathrate_df[val]:
        s=str(i)
        s1=dtype(s.replace("+","").replace(",","").replace("%","").replace("years old", "").replace("no fatalities","0.0").strip())
        records_list.append(s1)
    return records_list

updated_age = records_cleaner("Age",str)
updated_death_rate_cl = records_cleaner("Death Rate (Confirmed)",float)
updated_death_rate_acl = records_cleaner("Death Rate (All Cases)",float)
updated_deathrate_df = pd.DataFrame(list(zip(updated_age,updated_death_rate_cl, updated_death_rate_acl)), columns=["Age","Death Rate (Confirmed)","Death Rate (All Cases)"])
 

updated_deathrate_df


,Age,Death Rate (Confirmed),Death Rate (All Cases)
0,80,21.9,14.8
1,70-79,0.0,8.0
2,60-69,0.0,3.6
3,50-59,0.0,1.3
4,40-49,0.0,0.4
5,30-39,0.0,0.2
6,20-29,0.0,0.2
7,10-19,0.0,0.2
8,0-9,0.0,0.0


#### TIME SERIES DATA
##### Source: Johns Hopkins CSSE

In [536]:
ts_confirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
ts_confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,634,663,678,712,726,736,750,766,773,782
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2910,3007,3127,3256,3382,3517,3649,3848,4006,4154
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,723,723,731,738,738,743,743,743,745,745
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,25,25,25,25,26,27,27,27,27,30


In [537]:
# Data Cleaning 
# Replacing Missing Values or NaN with Zero
detect_missing_val(ts_confirmed_df)

Replaced missing values with Zero for : Country/Region
Replaced missing values with Zero for : Lat
Replaced missing values with Zero for : Long
Replaced missing values with Zero for : 1/22/20
Replaced missing values with Zero for : 1/23/20
Replaced missing values with Zero for : 1/24/20
Replaced missing values with Zero for : 1/25/20
Replaced missing values with Zero for : 1/26/20
Replaced missing values with Zero for : 1/27/20
Replaced missing values with Zero for : 1/28/20
Replaced missing values with Zero for : 1/29/20
Replaced missing values with Zero for : 1/30/20
Replaced missing values with Zero for : 1/31/20
Replaced missing values with Zero for : 2/1/20
Replaced missing values with Zero for : 2/2/20
Replaced missing values with Zero for : 2/3/20
Replaced missing values with Zero for : 2/4/20
Replaced missing values with Zero for : 2/5/20
Replaced missing values with Zero for : 2/6/20
Replaced missing values with Zero for : 2/7/20
Replaced missing values with Zero for : 2/8/20


In [538]:
ts_confirmed_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20
0,0,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335
1,0,Albania,41.153300,20.168300,0,0,0,0,0,0,...,634,663,678,712,726,736,750,766,773,782
2,0,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,2910,3007,3127,3256,3382,3517,3649,3848,4006,4154
3,0,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,723,723,731,738,738,743,743,743,745,745
4,0,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,25,25,25,25,26,27,27,27,27,30
5,0,Antigua and Barbuda,17.060800,-61.796400,0,0,0,0,0,0,...,24,24,24,24,24,24,24,24,24,25
6,0,Argentina,-38.416100,-63.616700,0,0,0,0,0,0,...,3144,3435,3607,3780,3892,4003,4127,4285,4428,4532
7,0,Armenia,40.069100,45.038200,0,0,0,0,0,0,...,1473,1523,1596,1677,1746,1808,1867,1932,2066,2148
8,Australian Capital Territory,Australia,-35.473500,149.012400,0,0,0,0,0,0,...,104,104,105,106,106,106,106,106,106,106
9,New South Wales,Australia,-33.868800,151.209300,0,0,0,0,3,4,...,2971,2976,2982,2994,3002,3004,3016,3016,3025,3030


## Consolidated Findings

#### Age of Coronavirus Deaths 
Source : [New York City Health](https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-deaths-04152020-1.pdf)

Website : https://www.worldometers.info/coronavirus/coronavirus-age-sex-demographics/

- Death Rate = (number of deaths / number of cases) = probability of dying if infected by the virus (%)

In [523]:
updated_deathrate_df

,Age,Death Rate (Confirmed),Death Rate (All Cases)
0,80,21.9,14.8
1,70-79,0.0,8.0
2,60-69,0.0,3.6
3,50-59,0.0,1.3
4,40-49,0.0,0.4
5,30-39,0.0,0.2
6,20-29,0.0,0.2
7,10-19,0.0,0.2
8,0-9,0.0,0.0


#### Genderwise Segmentation
- PC  : Pre-existing Conditions
- WPC : Without Pre-existing Conditions
- UPC : Unknown Pre-existing Conditions

In [524]:
genderwise_df

,Gender,Deaths,Share_of_Deaths(%),PC,Share_of_PC(%),WPC,Share_of_WPC(%),UPC,Share_of_UPC(%)
0,Male,4095,618.0,3087.000,622.0,96.0,72.2,912.0,59.5
1,Female,2530,38.2,1.873,37.8,37.0,27.8,620.0,40.5


#### Agewise Segmentation
- PC  : Pre-existing Conditions
- WPC : Without Pre-existing Conditions
- UPC : Unknown Pre-existing Conditions

In [525]:
agewise_df

,Age Groups,Deaths,Share of Deaths(%),PC,WPC,UPC
0,0 - 17,3,0.04,3,0.0,0.0
1,18 - 44,309,4.50,244,25.0,40.0
2,45 - 64,1581,23.10,1343,59.0,179.0
3,65 - 74,1683,24.60,1272,26.0,385.0


#### Total Agewise Segmentation
- TPC  : Total Pre-existing Conditions
- TWPC : Total Without Pre-existing Conditions
- TUPC : Total Unknown Pre-existing Conditions

In [527]:
total_agewise_df

,Total Deaths,Total Share of Deaths(%),TPC,TWPC,TUPC
0,6839,100,5151,137,1551


#### Corona Cases by Country

In [528]:
df_world.head()

,Country,Total_Cases,New_Cases
0,USA,1121712,26689
1,Spain,242988,3648
2,Italy,207428,1965
3,UK,177454,6201
4,France,167346,168


#### Corona Cases by State [INDIA]

In [529]:
df_india.head()

,States,Confirmed_Cases,Cured_Cases,Death_Cases
0,Andaman and Nicobar Islands,33,16,0
1,Andhra Pradesh,1463,403,33
2,Arunachal Pradesh,1,1,0
3,Assam,42,29,1
4,Bihar,426,82,2


#### Time Series records for Countrywise Corona Cases

In [539]:
ts_confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20
0,0,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335
1,0,Albania,41.1533,20.1683,0,0,0,0,0,0,...,634,663,678,712,726,736,750,766,773,782
2,0,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2910,3007,3127,3256,3382,3517,3649,3848,4006,4154
3,0,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,723,723,731,738,738,743,743,743,745,745
4,0,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,25,25,25,25,26,27,27,27,27,30
